# Greenplum Airbyte Connector - Python prototype

This notebook is used to validate a connection to greenplum cluster using python

In [ ]:
pip install psycopg2 pandas

In [16]:
import io
import pandas as pd
import psycopg2
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

def executeQueryInGreenplum(gp_cursor, query):
    gp_cursor.execute(query)
    # gp_cursor.commit()
    response_records = gp_cursor.fetchall()

    print("Print each row and it's columns values")
    for row in response_records:
        print(row)


# setup variables for the job
greenplum_host='54.89.84.45'
greenplum_database='eon_collective'
greenplum_user='eon_heimdal_proxy'
greenplum_password='0imb6vkiGQIvVrY'
greenplum_tableName='pandas_tbl'
greenplum_SchemaName= 'public'

# initialize dummy data with a list of lists- commas separated values, each sublist is representative of a dataframe row
data = [['tom', 10], ['nick', 15], ['juli', 14]]
 
# Create a pandas DataFrame named `df``
df = pd.DataFrame(data, columns=['Name', 'Age'])
 
# print dataframe.
print(df)

# Write your dataframe to memory as csv
csv_io = io.StringIO()
df.to_csv(csv_io, sep='\t', header=False, index=False)
csv_io.seek(0)


# Connect to the GreenPlum database.
greenplum = psycopg2.connect(host=greenplum_host, database=greenplum_database, user=greenplum_user, password=greenplum_password, port='5000')
gp_cursor = greenplum.cursor()




# create a table in greenplum
ddl_stmnt='''
create table %s.%s( name VARCHAR(255),age INTEGER)
DISTRIBUTED BY (name);'''%(greenplum_SchemaName, greenplum_tableName)

## execute the ddl statement 
executeQueryInGreenplum(gp_cursor, ddl_stmnt)


# # Copy the data from the buffer to the table.
# gp_cursor.copy_from(csv_io, '%s.%s'%(greenplum_database, greenplum_tableName))
# greenplum.commit()

# Close the GreenPlum cursor and connection.
gp_cursor.close()
greenplum.close()

   Name  Age
0   tom   10
1  nick   15
2  juli   14
